## Libraries and Dependencies

In [12]:
import torch
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.stats import norm
from pyDOE import lhs
# import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import akshare as ak
import yfinance as yf
import pandas as pd
from arch import arch_model


import warnings
warnings.filterwarnings('ignore')

np.random.seed(1234)

In [13]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Physics-informed Neural Networks

In [14]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1 #神经网络的层数
        
        # set up layer order dict
        self.activation = torch.nn.Tanh #激活函数为tanh
        
        layer_list = list() # 创建一个空列表 layer_list，用于存储神经网络的层和激活函数。
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        ) # 向 layer_list 添加输出层
        layerDict = OrderedDict(layer_list) #存储按顺序排列的层和激活函数
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [15]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self, X, u, layers, lb, ub ,r):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.S = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)

        self.r = r
        
        # settings 让参数lamdba_1 和 lambda_2 在训练过程中更新
        self.lambda_2 = torch.tensor([0.5], requires_grad=True).to(device)
        
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        self.dnn.register_parameter('lambda_2', self.lambda_2)
        
         # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters()) #参数更新
        self.iter = 0
        
    def net_u(self, S, t):  
        u = self.dnn(torch.cat([S, t], dim=1))
        return u
    
    def net_f(self, S, t):
        """ The pytorch autograd version of calculating residual """
        lambda_2 = torch.exp(self.lambda_2) #保证参数始终为正 σ > 0
        u = self.net_u(S, t)
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_S = torch.autograd.grad(
            u, S, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_SS = torch.autograd.grad(
            u_S, S, 
            grad_outputs=torch.ones_like(u_S),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = u_t + 0.5 * lambda_2 ** 2 * S ** 2 * u_SS + self.r * S * u_S - self.r * u
        return f
    
    def loss_func(self):
        u_pred = self.net_u(self.S, self.t)
        f_pred = self.net_f(self.S, self.t)
        loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
        self.optimizer.zero_grad()
        loss.backward()
        
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Loss: %e, l1: %.5f, l2: %.5f' % 
                (
                    loss.item(), 
                    torch.exp(self.lambda_2.detach()).item()
                )
            )
        return loss
    
    def train(self, nIter):
        self.dnn.train()
        for epoch in range(nIter):
            u_pred = self.net_u(self.S, self.t)
            f_pred = self.net_f(self.S, self.t)
            loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
            
            # Backward and optimize
            self.optimizer_Adam.zero_grad()
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 100 == 0:
                print(
                    'It: %d, Loss: %.3e, Lambda_2: %.6f' % 
                    (
                        epoch, 
                        loss.item(), 
                        # self.lambda_1.item(), 
                        torch.exp(self.lambda_2).item()
                    )
                )
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)
    
    def predict(self, X):
        S = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(S, t)
        f = self.net_f(S, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

## Configurations

In [16]:
option_sse_daily_sina_df = ak.option_sse_daily_sina(symbol='10006687')
option_sse_daily_sina_df = option_sse_daily_sina_df.drop_duplicates()
option_sse_daily_sina_df['日期'] = pd.to_datetime(option_sse_daily_sina_df['日期']).dt.strftime('%Y-%m-%d')

df = ak.fund_etf_hist_sina(symbol="sh510300")
df["date"] = pd.to_datetime(df["date"])
start_date = "2024-01-18"
end_date = "2024-06-26"
mask = (df["date"] >= start_date) & (df["date"] <= end_date)
filtered_df = df.loc[mask].reset_index(drop=True)
filtered_df['date'] = filtered_df['date'].dt.strftime('%Y-%m-%d')
hist = filtered_df[['date', 'close']]
hist.columns = ['日期', '股票收盘价']

data = pd.merge(option_sse_daily_sina_df, hist, on='日期', how='inner')
data = data.sort_values('日期').reset_index(drop=True)

In [17]:
train_data = data[['日期', '收盘', '股票收盘价']]

# 将日期转换为时间戳（数值类型）
train_data['日期'] = pd.to_datetime(train_data['日期']).astype(np.int64) // 10**9  # 转换为秒级时间戳


# 将数据分为特征（日期、期权收盘价）和目标（股票收盘价）
u_market = train_data['收盘'].values  
t_market = train_data['日期'].values  
S_market = train_data['股票收盘价'].values  


In [18]:
# 拼接市场数据
X_star = np.hstack((S_market[:, None], t_market[:, None]))
u_star = u_market[:, None]

# 归一化数据
lb = X_star.min(0)
ub = X_star.max(0)
X_train = (X_star - lb) / (ub - lb)  # 归一化资产价格和时间
u_train = (u_star - u_star.min()) / (u_star.max() - u_star.min())  # 归一化期权价格

In [19]:
# 参数范围
# S_max = 100  # 资产价格最大值
# T = 1.0      # 到期时间
K = 2.9       # 执行价格
r = 0.0141   # 无风险利率
sigma = 0.6224  # 波动率

# 初始化 PINN
layers =[2, 30, 30, 30, 30, 30, 30, 30, 1]

## Training on Rael Market Data

In [20]:
%%time

# training
model = PhysicsInformedNN(X_train, u_train, layers, lb, ub,r)
model.train(10000)

It: 0, Loss: 3.447e-01, Lambda_2: 1.650358
It: 100, Loss: 1.296e-02, Lambda_2: 1.471033
It: 200, Loss: 6.326e-04, Lambda_2: 1.428199
It: 300, Loss: 5.578e-04, Lambda_2: 1.425340
It: 400, Loss: 5.070e-04, Lambda_2: 1.422583
It: 500, Loss: 4.876e-04, Lambda_2: 1.420023
It: 600, Loss: 4.576e-04, Lambda_2: 1.417917
It: 700, Loss: 4.394e-04, Lambda_2: 1.415412
It: 800, Loss: 4.403e-04, Lambda_2: 1.412911
It: 900, Loss: 4.204e-04, Lambda_2: 1.411397
It: 1000, Loss: 4.146e-04, Lambda_2: 1.408841
It: 1100, Loss: 4.088e-04, Lambda_2: 1.407665
It: 1200, Loss: 4.053e-04, Lambda_2: 1.404903
It: 1300, Loss: 4.016e-04, Lambda_2: 1.403896
It: 1400, Loss: 5.213e-04, Lambda_2: 1.401348
It: 1500, Loss: 3.975e-04, Lambda_2: 1.399583
It: 1600, Loss: 3.960e-04, Lambda_2: 1.398776
It: 1700, Loss: 4.012e-04, Lambda_2: 1.394600
It: 1800, Loss: 3.945e-04, Lambda_2: 1.393612
It: 1900, Loss: 3.939e-04, Lambda_2: 1.392914
It: 2000, Loss: 4.724e-04, Lambda_2: 1.388883
It: 2100, Loss: 3.935e-04, Lambda_2: 1.386994


In [21]:
# evaluations
# u_pred, f_pred = model.predict(X_star)

# error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)

# U_pred = griddata(X_star, u_pred.flatten(), (S_grid, t_grid), method='cubic')

lambda_2_value = model.lambda_2.detach().cpu().numpy()
lambda_2_value = np.exp(lambda_2_value)

error_lambda_2 = np.abs(lambda_2_value - sigma) / sigma * 100

print('lambda_2:', lambda_2_value)

# print('Error u: %e' % (error_u))                         
print('Error l2: %.5f%%' % (error_lambda_2))  

lambda_2: [1.2900102]
Error l2: 107.26385%
